In [ ]:
import os
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from matplotlib import patches as mpatches
import librosa
from scipy import signal as scipysig 
from scipy.signal import find_peaks
from scipy.stats import gaussian_kde
from natsort import natsorted
from matplotlib.animation import FuncAnimation

In [ ]:
# Setting parameters
recordings = 26
dir = '/Users/lillianwang/Documents/bird-counts-25/'
birdnet_dir = dir+'data/birdnet/'
ebird_dir = dir+'data/ebird/'

In [ ]:
def load_data(dir):
    files = natsorted([f for f in os.listdir(dir) if f.endswith('.csv')])
    dfs = []
    for i, file in enumerate(files):
        df = pd.read_csv(dir+file, encoding='latin1')
        df['Recording'] = i+1
        dfs.append(df)
    return dfs

In [ ]:
# Read data
birdnet_dfs = load_data(birdnet_dir)
ebird_dfs = load_data(ebird_dir)

# Strip durations to digits
def extract_numbers(text):
    return ''.join(filter(str.isdigit, text))
    
for ebird_df, birdnet_df in zip(ebird_dfs, birdnet_dfs):
    if not ebird_df.empty:
        birdnet_df['Duration'] = ebird_df['Duration'].apply(extract_numbers)
        ebird_df['Duration'] = ebird_df['Duration'].apply(extract_numbers)

combined_birdnet = pd.concat(birdnet_dfs)
combined_ebird = pd.concat(ebird_dfs)

## Helper functions

In [ ]:
# Setting parameters
n_fft = 1024
nf = 160
highpass_filter = 500

In [ ]:
# ---- filtering function ----
def butter_highpass_filter(data, cut, fs, order=5):
    cutoff = 2 * cut / fs
    b, a = scipysig.butter(order, cutoff, btype='high', analog=False)
    y = scipysig.filtfilt(b, a, data)
    return y

# ---- function to convert linear units to dB ----
def logTransform(spec, scale=10**(-5)):
    return 20 * np.log10(spec + scale * np.ones(spec.shape))

# ---- function to load 4-channel audio and convert to B-format ----
def loadData(uploaded_file):
    # ---- read in file ----
    (s, framerate) = librosa.core.load(uploaded_file, sr=None, mono=False)

    # ---- get part of signal ----
    s = s/np.max(s)**2

    # ---- filter signal ----
    s = butter_highpass_filter(s, highpass_filter, framerate)
    s = s[:4, :]
    # ---- convert to b-format through matrix multiplication ----
    b_transform = np.asarray([[1, 1, 1, 1], [1, 1, -1, -1], [1, -1, 1, -1], [1, -1, -1, 1]])
    s_B = b_transform @ s

    return s, s_B, framerate

In [ ]:
# Load in file
recording = 15
target_species = 'Red-eyed Vireo'
duration = 600
bin_num = 60
file = os.path.join(dir+f'data/recordings/{recording}.WAV')
s, s_B, framerate = loadData(file)

In [ ]:
# make b-format spectrogram
def spectrogram(file, s, s_B, framerate, start, end):
  end = int(np.ceil(end * framerate))
  if start is not None:
      start = int(np.trunc(start * framerate))
      s = s[:, start:end]

  b_transform = np.asarray([[1, 1, 1, 1], [1, 1, -1, -1], [1, -1, 1, -1], [1, -1, -1, 1]])
  s_B = b_transform @ s

  specs = []
  for num in np.arange(4):
      freqs, inds, spec = scipysig.stft(s_B[num,:], fs=framerate, nperseg=n_fft)
      nf_full = len(freqs)
      freqs = freqs[0:nf]
      specs.append(spec[0:nf, :].T)

  # directly get the three components
  w = specs[0] # p(f,t)
  x = specs[1] # v(f,t) x
  y = specs[2] # v(f,t)
  # azimuth values for all pixels
  azimuth = np.arctan2(np.real(w.conj() * y), np.real(w.conj() * x)) # eq 10

  # weight the azimuth values by the intensity
  weights = np.abs(w)**2

  # get grids for time and frequency
  f_grid, time_grid = np.meshgrid(freqs, inds)

  # need to set these parameters for histogram
  duration = len(s_B[0])/framerate
  time_step = 0.05
  num_time = int(duration * 1/time_step)
  num_azim = 60

  # histogram
  hist, azim_edges, time_edges = np.histogram2d(x = azimuth.ravel(), y = time_grid.ravel(),
                                                bins=[num_azim, num_time],
                                                weights = weights.ravel())

  log_hist = np.log(hist + 0.01 * np.ones(hist.shape))

  return azimuth, weights, freqs, hist, azim_edges, time_edges, log_hist

In [ ]:
# Find timestamps of audio segments containing target vocalization
def generate_timestamps(recording, target_species=None, start_time=0, end_time=duration):
    df = birdnet_dfs[recording-1]

    if target_species is None:
      birdnet_rows = df
    else:
      birdnet_rows = df[df['Common name'] == target_species]

    timestamps = []
    species = []
    confidence = []

    for i in range(len(birdnet_rows)):
      start = birdnet_rows.iloc[i]['Start (s)']
      end = birdnet_rows.iloc[i]['End (s)']
      if start >= start_time and end <= end_time:
        timestamps.append((start, end))
        confidence.append(birdnet_rows.iloc[i]['Confidence'])
        if target_species is None:
          species.append(birdnet_rows.iloc[i]['Common name'])
      else:
        continue

    return timestamps, species, confidence

In [ ]:
# Absolute peak of histogram
def absolute_peaks(azimuth, weights):
  counts, bin_edges = np.histogram(azimuth, weights=weights, bins=bin_num)
  peaks, _ = find_peaks(counts, height=np.max(counts))
  bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
  return counts, bin_centers[peaks]

# Calculate histogram peaks using percent of peak method
def histogram_peaks(azimuth, weights):
  counts, bin_edges = np.histogram(azimuth, weights=weights, bins=bin_num)
  peaks, _ = find_peaks(counts, prominence=0.5*np.max(counts))
  bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
  return counts, bin_centers[peaks]

# Calculate histogram peaks using kernel density estimation method
def kde_peaks(azimuth, weights):
  kde = gaussian_kde(azimuth, weights=weights)
  x_kde = np.linspace(np.min(azimuth), np.max(azimuth))
  y_kde = kde(x_kde)

  kde_peaks_indices, _ = find_peaks(y_kde, prominence=0.01)
  kde_peaks = x_kde[kde_peaks_indices]
  return kde_peaks, x_kde, y_kde

In [ ]:
# Filter out azimuths outside range [low_freq, high_freq]
def azimuth_filter(azimuth, weights, freqs, low_freq, high_freq):
    freq_range = (freqs >= low_freq) & (freqs <= high_freq)

    filtered_azimuths = azimuth[:, freq_range]
    filtered_weights = weights[:, freq_range]
    filtered_freqs = freqs[freq_range]

    return filtered_azimuths, filtered_weights, filtered_freqs

## Azimuth

In [ ]:
# Combined azimuth over time
plt.figure(figsize=(12, 9))

x_vals = []
y_vals = []
sizes = []
colors = []

timestamps, species, confidence = generate_timestamps(recording)

species_list = sorted(set(species))  # Consistent order
species_to_color = {sp: i for i, sp in enumerate(species_list)}

for i in range(len(timestamps)):
  start, end = (timestamps[i][0], timestamps[i][1])
  azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, start, end)
  azimuth = azimuth.flatten()
  weights = weights.flatten()

  counts, peaks = absolute_peaks(azimuth, weights)
  #peaks, x_kde, y_kde = kde_peaks(azimuth, weights)
  #counts, peaks = histogram_peaks(azimuth, weights)

  for peak in peaks:
    x_vals.append(start)
    y_vals.append(peak)
    sizes.append(confidence[i]*1000)
    colors.append(species_to_color[species[i]])

plt.scatter(x_vals, y_vals, c=colors, cmap='turbo', s=sizes, alpha=.75)
cmap = plt.cm.turbo
num_species = len(species_list)
patches = [mpatches.Patch(color=cmap(species_to_color[sp]/(num_species-1)), label=sp)
           for sp in species_list]

plt.title('Azimuth over time')
plt.xlabel('Time (s)')
plt.ylabel('Azimuth')
plt.ylim(-np.pi, np.pi)
plt.grid(True)
plt.legend(handles=patches, title='Species',loc=(1.05, 0))

plt.tight_layout()
plt.savefig('time-vs-azimuth-combined.png', dpi=200)
plt.show()


In [ ]:
# Histograms of annotations
timestamps, species, confidence = generate_timestamps(recording, target_species)
fig, axes = plt.subplots(nrows=len(timestamps), ncols=1, figsize=(8, len(timestamps)*4))

x_vals = []
y_vals = []
sizes = []

for i in range(len(timestamps)):
  start, end = (timestamps[i][0], timestamps[i][1])
  azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, start, end)

  azimuth = azimuth.flatten()
  weights = weights.flatten()

  counts, peaks = absolute_peaks(azimuth, weights)
  #peaks, x_kde, y_kde = kde_peaks(azimuth, weights)
  #counts, peaks = histogram_peaks(azimuth, weights)

  axes[i].hist(azimuth, weights=weights, bins=bin_num, color='b', density=True)
  ymin, ymax = axes[i].get_ylim()
  axes[i].vlines(peaks, ymin=0, ymax=ymax, colors='red', linestyles='dashed', linewidth=0.75)
  # axes[i].plot(x_kde, y_kde, linewidth=.75, label='KDE')
  
  axes[i].set_title(f'{target_species} weighted histogram of azimuth ({recording}, {start:.0f}-{end:.0f}s)')
  axes[i].set_xlabel('Azimuth (radians)', fontsize=10)
  #print(f'{recording}.{i+1} azimuth peaks: {peaks}')

plt.tight_layout()
plt.savefig(f'{target_species}-azimuth-histograms.png', dpi=200)
plt.show()

In [ ]:
# Azimuth over time from detections (size weighed by confidence)
plt.figure(figsize=(8, 6))

x_vals = []
y_vals = []
sizes = []

timestamps, species, confidence = generate_timestamps(recording, target_species)

for i in range(len(timestamps)):
  start, end = (timestamps[i][0], timestamps[i][1])
  azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, start, end)
  azimuth = azimuth.flatten()
  weights = weights.flatten()

  counts, peaks = absolute_peaks(azimuth, weights)
  #peaks, x_kde, y_kde = kde_peaks(azimuth, weights)
  #counts, peaks = histogram_peaks(azimuth, weights)

  for peak in peaks:
    x_vals.append(start)
    y_vals.append(peak)
    sizes.append(confidence[i]*500)

plt.scatter(x_vals, y_vals, s=sizes, alpha=.75, color='b')
plt.ylim(-np.pi, np.pi)
plt.grid(True)

plt.title(f'{target_species} azimuth over time')
plt.xlabel('Time (s)')
plt.ylabel('Azimuth')

plt.tight_layout()
plt.savefig(f'{target_species}-time-vs-azimuth.png', dpi=200)
plt.show()


In [ ]:
# Azimuth over time (3s windows with 1.5s overlap)
plt.figure(figsize=(8, 6))

x_vals = []
y_vals = []
timestamps = []
ind = 0
while ind < duration:
  timestamps.append((ind, ind+3))
  ind += 3

for timestamp in timestamps:
  start, end = timestamp[0], timestamp[1]
  azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, start, end)
  azimuth = azimuth.flatten()
  weights = weights.flatten()

  counts, peaks = absolute_peaks(azimuth, weights)
  #peaks, x_kde, y_kde = kde_peaks(azimuth, weights)
  #counts, peaks = histogram_peaks(azimuth, weights)

  for peak in peaks:
    x_vals.append(start)
    y_vals.append(peak)

plt.scatter(x_vals, y_vals, alpha=.75, color='b')
plt.ylim(-np.pi, np.pi)
plt.grid(True)

plt.title('Azimuth over time')
plt.xlabel('Time (s)')
plt.ylabel('Azimuth')

plt.tight_layout()
plt.savefig('time-vs-azimuth-scatter.png', dpi=200)
plt.show()


In [ ]:
# Azigram over time
azimuth, weights, freqs, hist, azim_edges, time_edges, log_hist = spectrogram(file, s, s_B, framerate, 10, 600)

#log_hist = np.log(hist + 1e-6 * np.ones(hist.shape))
fig_hist, ax_hist = plt.subplots(1, 1, figsize=(14, 6))
ax_hist.pcolormesh(time_edges[0:-1], 180/np.pi * azim_edges[0:-1], log_hist, cmap='turbo')
ax_hist.set_xlim([10, 20])
ax_hist.set_ylim([-180, 180])

ax_hist.set_title('Histogram over azimuth and time', fontsize=16)
ax_hist.set_xlabel('Time (sec)', fontsize=14)
ax_hist.set_ylabel('Azimuth (degrees)', fontsize=14)

fig_hist.savefig('time-vs-azigram.png', dpi=200)

In [ ]:
# Unweighted cumulative polar plot
plt.figure(figsize=(8, 8))

azimuth, *_ = spectrogram(file, s, s_B, framerate, 10, 600)
counts, bin_edges = np.histogram(azimuth.flatten(), bins=bin_num)
bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])

ax = plt.subplot(1, 1, 1, polar=True)
ax.plot(bin_centers, counts, color='b')

ax.set_title('Unweighted azimuth')
ax.set_xlabel('Azimuth (degrees)')

plt.savefig('polar-unweighted-azimuth.png', dpi=200)
plt.show()


In [ ]:
# Weighted cumulative polar plot
plt.figure(figsize=(8, 8))

azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, 10, 600)
counts, bin_edges = np.histogram(azimuth.flatten(), weights=weights.flatten(), bins=bin_num)
bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])

ax = plt.subplot(1, 1, 1, polar=True)
ax.plot(bin_centers, counts, color='b')

ax.set_title('Weighted azimuth')
ax.set_xlabel('Azimuth (degrees)')

plt.savefig('polar-weighted-azimuth.png', dpi=200)
plt.show()


In [ ]:
# # Azimuth animation over time
# fig = plt.figure(figsize=(8, 8))
# ax = plt.subplot(1, 1, 1, polar=True)
# ax.set_title('Weighted azimuth', pad=30)
# ax.set_xlabel('Azimuth (radians)')

# azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, 9, 10)
# counts, bin_edges = np.histogram(azimuth.flatten(), weights=weights.flatten(), bins=bin_num)
# bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# line, = ax.plot(bin_centers, counts)

# timestamps = []
# ind = 10
# step = 2
# while ind < duration:
#   timestamps.append((ind, ind+step))
#   ind += step//2

# azihist = []
# for start, end in timestamps:
#   azimuth, weights, *_ = spectrogram(file, s, s_B, framerate, start, end)
#   counts, bin_edges = np.histogram(azimuth.flatten(), weights=weights.flatten(), bins=bin_num)
#   bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
#   azihist.append((bin_centers, counts))

# def update(frame):
#     line.set_data(azihist[frame][0], azihist[frame][1])
#     ax.set_ylim(0, azihist[frame][1].max()*1.05)
#     ax.set_title(f'Azimuth\n{timestamps[frame][0]}s', pad=30)
#     return line,

# ani = FuncAnimation(fig, update, frames=len(timestamps),
#                     blit=False, interval=5000, repeat=False)

# ani.save('animation.mp4', fps=1, dpi=200)
# plt.show()
